In [1]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import copy as cp

nr = 500 # carrying capacity of rabbits
r_init = 100 # initial rabbit population
mr = 0.03 # magnitude of movement of rabbits
dr = 1.0 # death rate of rabbits when it faces foxes
rr = 0.1 # reproduction rate of rabbits
grate = 0.3
f_init = 30 # initial fox population
g_init = 100 #initial grass
mf = 0.05 # magnitude of movement of foxes
df = 0.1 # death rate of foxes when there is no food
df_rab = 0.05 #death rate of rabbits without food
rf = 0.5 # reproduction rate of foxes
cd = 0.04 # radius for collision detection
cdsq = cd ** 2

class agent:
    pass

def initialize():
    global agents, rdata, fdata, gdata
    agents = []
    for i in range(r_init + f_init + g_init):
        ag = agent()
        if i < r_init: 
            ag.type = 'r' 
        if i >= r_init and i < r_init + f_init:
            ag.type = 'f'  
        else:
            ag.type = 'g'
        ag.x = random()
        ag.y = random()
        agents.append(ag)
    
    rdata = []
    fdata = []
    gdata = []
        
def observe():
    
    global agents, rdata, fdata, gdata
    subplot(2, 1, 1)
    cla()
    rabbits = [ag for ag in agents if ag.type == 'r']
    rdata.append(len(rabbits))
    if len(rabbits) > 0:
        x = [ag.x for ag in rabbits]
        y = [ag.y for ag in rabbits]
        plot(x, y, 'b.')
    foxes = [ag for ag in agents if ag.type == 'f']
    fdata.append(len(foxes))
    if len(foxes) > 0:
        x = [ag.x for ag in foxes]
        y = [ag.y for ag in foxes]
        plot(x, y, 'ro')
    
    grass = [ag for ag in agents if ag.type == 'g']
    gdata.append(len(foxes))
    if len(grass) > 0:
        x = [ag.x for ag in grass]
        y = [ag.y for ag in grass]
        plot(x, y, 'g.')
    axis('image')
    axis([0, 1, 0, 1])
    subplot(2, 1, 2)
    cla()
    plot(rdata, label = 'prey')
    plot(fdata, label = 'predator')
    #plot(gdata, label = 'grass')
    legend()


def update():
    global agents
    if agents == []:
        return
    ag = agents[randint(len(agents))]
    
    # simulating random movement
    m = mr if ag.type == 'r' else mf
    ag.x += uniform(-m, m)
    ag.y += uniform(-m, m)
    
    ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x
    ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y
    
    # detecting collision and simulating death or birth
    neighborsRBw = [nb for nb in agents if nb.type == 'f'
               and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < cdsq]
    neighborsRBg = [nb for nb in agents if nb.type == 'g'
               and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < cdsq]
    neighborsGr = [nb for nb in agents if nb.type == 'r'
               and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < cdsq]
    neighborsWr = [nb for nb in agents if nb.type == 'r'
               and (ag.x - nb.x)**2 + (ag.y - nb.y)**2 < cdsq]
    
    
    
    
    if ag.type == 'r':
        if len(neighborsRBw) > 0: # if there are foxes nearby
            agents.remove(ag)
            #print("okay")
        if random() < rr*(1-sum(1 for x in agents if x.type == 'r')/nr):
            agents.append(cp.copy(ag))
        if len(neighborsRBg) > 0: # if there are no grass nearby
            if random() < df_rab:
                agents.remove(ag)
            
    elif ag.type == 'f':
        if len(neighborsWr) != 0: # if there are rabbits nearby
            agents.remove(neighborsWr)
            if random() < rf:
                agents.append(cp.copy(ag))
                return
        else: # if there aren't rabbits nearby
            if random() < df:
                agents.remove(ag)

    elif ag.type == 'g':
        if len(neighborsGr) > 0: # if there are no rabbits nearby
            agents.remove(ag)

        if random() < grate:
            ag.x = random()
            ag.y = random()
            agents.append(cp.copy(ag))

                
def update_one_unit_time():
    global agents
    t = 0
    while t < 1.:
        t += 1 / len(agents)
        update()
    
import pycxsimulator
pycxsimulator.GUI().start(func=[initialize, observe, update_one_unit_time])

/Users/bswanberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:46: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/Users/bswanberg/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:69: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/bswanberg/anaconda/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args